In [ ]:
#import packages


In [ ]:
#system packages

import sys
import warnings
import os 
import traceback #obs? 
if not sys.warnoptions:
    warnings.filterwarnings("once")  

In [ ]:
#base packages:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats, integrate, optimize
import math
import datetime
from datetime import date, timedelta

In [ ]:
#optional packages

from statsmodels.tsa.base.datetools import dates_from_str
from IPython.display import display
pd.options.display.max_columns = 50  #allow DF.head to show all columns in notebook
from see import see
from tabulate import tabulate 
#from io import StringIO

In [ ]:
#packages for the econometrics / models

from statsmodels.tsa.vector_ar import vecm
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import DFGLS, ADF, KPSS, PhillipsPerron, ZivotAndrews
from arch.unitroot.cointegration import engle_granger, phillips_ouliaris
import statsmodels.formula.api as smf  #VAR package contained within 
import statsmodels.tsa.api as smt

#import the functionality for detecting mathematical errors (E.G. types of linear algebra issues etc.)
from statsmodels.tools.sm_exceptions import ValueWarning
from arch.utility.exceptions import (
    InfeasibleTestException,
    InvalidLengthWarning,
    invalid_length_doc)
warnings.filterwarnings("once", category = ValueWarning)

In [ ]:

#select KEY
gvkey_1408 = sorted_alldata_df_final.loc[sorted_alldata_df_final.gvkey==1408]

gvkey_number = 1408
gvkey_frame = gvkey_1408

print("JOHANSENS COINT TEST with individual GVKEY NUMBER: " + str(gvkey_number))

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
data = pd.DataFrame(data=(cds, pecds)).T
#data = np.column_stack((cds, pecds))

model = vecm.select_order(data, maxlags=5)
optimal_lag_length = model.bic
    
vecm_cointeg_rank = vecm.select_coint_rank(data, det_order =1, k_ar_diff = optimal_lag_length, 
                                                  method='trace', signif=0.05)
        
print(vecm_cointeg_rank.summary())
print("Number of Cointegrating relationships:", vecm_cointeg_rank.rank)

# VAR REGRESSION

In [ ]:
df_gvkey_150937 = df_basis_coupons_month.loc[df_basis_coupons_month.gvkey==150937.0]

#specify individual series
cds_150937 = df_gvkey_150937['market_cds_spread']
pecds_150937 = df_gvkey_150937['pecds']      

#alternatively, some methods require that a DF be used to maintain column labels/references
gvkey_150937_solved = pd.DataFrame(data=(cds_150937, pecds_150937)).T

#VAR_MODEL: GVKEY==2136
var_model_gvkey_150937 = smt.VAR(gvkey_150937_solved)
res_model_gvkey_150937 = var_model_gvkey_150937.fit(maxlags=var_model_gvkey_150937.select_order(maxlags=5).bic)
print("max_lags:", var_model_gvkey_150937.select_order().bic)
print(res_model_gvkey_150937.summary())

In [ ]:
#Original VECM base: CDS/PECDS

gvkey_number = 1408
gvkey_frame = gvkey_1408
#coint_rels = 0

print("VECM MODEL OF GVKEY: " + str(gvkey_number))

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
vecm_data_base = pd.DataFrame(data=(cds, pecds)).T

vecm_model_base = vecm.select_order(vecm_data_base, maxlags=5)
optimal_lag_length_base = vecm_model_base.bic
    
vecm_cointeg_rank_base = vecm.select_coint_rank(vecm_data_base, det_order =1, k_ar_diff = optimal_lag_length_base, 
                                                  method='trace', signif=0.05)

if vecm_cointeg_rank_base.rank >= 1:
    coint_rels_base = 1
else:
    print("insufficient number of cointegrating relationships for VECM model")
    
vecm_base = vecm.VECM(vecm_data_base, k_ar_diff=vecm_model_base.bic, coint_rank=coint_rels_base, deterministic='cili')
vecm_base_res = vecm_base.fit()
print(vecm_base_res.summary())




vecm_base_res.test_granger_causality(caused=0).summary()

In [ ]:
#VECM of base + VIX: CDS/PECDS
#with automated coint rels calculation

gvkey_number = 1408
gvkey_frame = gvkey_1408
#coint_rels = 0

print("VECM MODEL + VIX OF GVKEY: " + str(gvkey_number))

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
vix = gvkey_frame['cboe_vix']
vecm_data_vix = pd.DataFrame(data=(cds, pecds, vix)).T

vecm_model_vix = vecm.select_order(vecm_data_vix, maxlags=5)
optimal_lag_length_vix = vecm_model_vix.bic
    
vecm_cointeg_rank_vix = vecm.select_coint_rank(vecm_data_vix, det_order =1, k_ar_diff = optimal_lag_length_vix, 
                                                  method='trace', signif=0.05)

#if vecm_cointeg_rank_vix.rank >= 1:
#    coint_rels_vix = 1
#else:
#    print("insufficient number of cointegrating relationships for VECM model")
    
vecm_vix_2 = vecm.VECM(vecm_data_vix, k_ar_diff=vecm_model_vix.bic, coint_rank=vecm_cointeg_rank_vix.rank, deterministic='cili')
vecm_vix_res_2 = vecm_vix_2.fit()
print(vecm_vix_res_2.summary())



vecm_vix_res_2.test_granger_causality(causing='cboe_vix', caused='market_cds_spread').summary()